## Modules à importer

In [3]:
import math
import scipy
import pickle
import numpy as np
import pandas as pd
import iminuit as im
import ipywidgets as ipw
from snprop import age
from snprop import survey
from variaIa import tools_ar
from variaIa.tools_ar import surveys, df_cons, df_nc, df_c, df_full
from variaIa import massevol
from importlib import reload
from variaIa import stretchevol
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
import random

## Définitions pandas, surveys et sélection de l'échantillon à utiliser

In [4]:
def set_survey(cons, su):
    global df
    if su == 'All':
        df = df_cons(cons)
    elif 'n' in su:
        df = df_cons(cons)[df_cons(cons)['survey'] != su.split('n')[-1]]
    else:
        df = df_cons(cons)[df_cons(cons)['survey'] == su]
    return df

int_set_su = ipw.interactive(set_survey, cons=tools_ar.cons, su=tools_ar.su)
display(int_set_su)

smap = plt.cm.get_cmap('cividis')
#smap = plt.cm.get_cmap('viridis')
colors = {'SNF': plt.cm.binary(0.9),
          'low-z': (1, 0, 0),
          'SDSS': smap(0.1),
          'PS1': smap(0.5),
          'SNLS': smap(0.8),
          'HST': smap(0.99),  # for cividis
          #'HST': smap(300),  # for viridis
          'All': (255/255, 51/255, 0/255)} 

def alphacolors(alpha, survey):
    return(colors[survey][0], colors[survey][1], colors[survey][2], alpha)

df_nHST = df_nc[df_nc['survey'] != 'HST']
df_nlowz = df_nc[df_nc['survey'] != 'low-z']

interactive(children=(Checkbox(value=False, description='Conservative'), Dropdown(description='Survey:', optio…

## Minimisation des modèles à mettre dans le _pandas_

In [8]:
gen = massevol.generic()
gen.set_model('Evol3G3M3S')
evol3G3M3S = gen.fit(df, limit_a=(0,1))

gen.set_model('Evol3G3M4S')
evol3G3M4S = gen.fit(df, limit_a=(0,1), limit_mu_2=(8, 10), limit_mu=(10, 11))

gen.set_model('Evol2G2M2S')
evol2G2M2S = gen.fit(df)

gen.set_model('Evol2G2M3S')
evol2G2M3S = gen.fit(df)

## Get proba

In [9]:
def get_proba(model):
    return np.exp((evol3G3M3S.get_aic() - model.get_aic())/2)

## Comparison dataframe

In [10]:
modèles = [evol3G3M3S, evol3G3M4S, evol2G2M2S, evol2G2M3S]

d_mod_comp = pd.DataFrame({'Name': [k.__doc__ for k in modèles],
                           'Fixed': [k.FIXED for k in modèles],
                           'Free param': [len(k.FREEPARAMETERS) for k in modèles],
                           '$\mathcal{L}$': [round(k.get_logl(),1) for k in modèles],
                           'AIC': [round(k.get_aic(),1) for k in modèles],
                           '$\Delta$ AIC': [round(evol3G3M3S.get_aic()-k.get_aic(),1) for k in modèles],
                           'Proba': [get_proba(k) for k in modèles]
                          })

path = '../../../Data/tables/mass/mass_df_comp'
if tools.cons.value:
    path += '_cons'
path += '.dat'
d_mod_comp.to_csv(path)

d_mod_comp

,Name,Fixed,Free param,$\mathcal{L}$,AIC,$\Delta$ AIC,Proba
0,3G3M3S,False,7,218.8,232.8,0.0,1.000000
1,3G3M4S,False,8,220.1,236.1,-3.4,0.184660
2,2G2M2S,False,4,230.0,238.0,-5.3,0.072142
3,2G2M3S,False,5,229.8,239.8,-7.1,0.029276
